In [1]:
import tweepy as tweepy

In [2]:
import pandas as pd

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import requests
import json

# Gether Data #

In [4]:
df = pd.read_csv('twitter-archive-enhanced.csv')

In [5]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
image_url = requests.get(url, allow_redirects=True)

with open('image_predictions .tsv', mode='wb')as file : 
    file.write(image_url.content)
    df2 = pd.read_csv('image-predictions .tsv',sep='\t')

ProxyError: HTTPSConnectionPool(host='d17h27t6h515a5.cloudfront.net', port=443): Max retries exceeded with url: /topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002C8C5947640>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')))

import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer

Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
#These are hidden to comply with Twitter's API terms and conditions
consumer_key = 'HIDDEN'
consumer_secret = 'HIDDEN'
access_token = 'HIDDEN'
access_secret = 'HIDDEN'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

#NOTE TO STUDENT WITH MOBILE VERIFICATION ISSUES:
#df_1 is a DataFrame with the twitter_archive_enhanced.csv file. You may have to
#change line 17 to match the name of your DataFrame with twitter_archive_enhanced.csv
#NOTE TO REVIEWER: this student had mobile verification issues so the following
#Twitter API code was sent to this student from a Udacity instructor
#Tweet IDs for which to gather additional data via Twitter's API
tweet_ids = df_1.tweet_id.values
len(tweet_ids)

#Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()
#Save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict)

In [ ]:
df3 = pd.read_json('tweet-json.txt', lines= True)

In [ ]:
df.head(3)

In [ ]:
df2

In [ ]:
df3.head(2)

In [ ]:
df.info

In [ ]:
df2.info

In [ ]:
df.info

# Data Cleaning - Quality #

## Define ##

•	I made a copy of each database to make sure that all the cleaning efforts are done in a separate data and the original is preserved.

•	Then, I noticed that the timestamp column is not easy to read because the date and time are merged together. I separated them into a date column and a time column using the datetime from Pandas.

•	After that, the database had three columns for date and time: date column, time column, and the original timestamp column. I dropped the timestamp column using the drop function 

•	I renamed the id column in df3_clean to tweet_id using the rename function to match the other databases because we will be merging the columns later.

•	I removed duplicate images in jpg_url column using duplicated() and drop_duplicates.

•	I rounded up the decimals in confidence rate columns to 2 decimals

•	I noticed that the names of the predicted dog breed are separated using “_” instead of spaces, so I replaced the _ in the predicted names to spaces.

•	I removed retweets 

•	I dropped replies columns


## Code ##

In [ ]:
#we first make a copy to clean for each database
df_clean = df.copy()
df2_clean = df2.copy()
df3_clean = df3.copy()

In [ ]:
#first data quality issue: timestamp column needs to be separated to date and time columns using pandas' datetime

In [ ]:
df_clean['Dates'] = pd.to_datetime(df_clean['timestamp']).dt.date
df_clean['Time'] = pd.to_datetime(df_clean['timestamp']).dt.time

In [ ]:
#second data quality issue: the database now has duplicate columns, so we should drop the timestamp column

In [ ]:
df_clean = df_clean.drop('timestamp', 1)

In [ ]:
df_clean.head(2)

In [ ]:
#third data quality issue: rename the id column in df3_clean to tweet_id to match the other databases
#because we will be merging the columns later

In [ ]:
df3_clean.rename(columns={'id': 'tweet_id'}, inplace=True)

In [ ]:
df3_clean.head(1)

In [ ]:
#fourth data quality issue: remove duplicate images in jpg_url column

In [ ]:
df2_clean['jpg_url'].duplicated().sum()

In [ ]:
df2_clean['jpg_url'].drop_duplicates(inplace=True)

In [ ]:
df2_clean.head(2)

In [ ]:
#fifth data quality issue: round the decimals in confidence rate columns to 2 decimals

In [ ]:
df2_clean.round({"p1_conf":2, "p2_conf":2, "p3_conf":2})

In [ ]:
#sixth data quality issue: replace the _ in the predicted names to spaces

In [ ]:
df2_clean['p1'] = df2_clean['p1'].str.replace('_',' ')
df2_clean['p2'] = df2_clean['p2'].str.replace('_',' ')
df2_clean['p3'] = df2_clean['p3'].str.replace('_',' ')

## Test ##

In [ ]:
df2_clean

In [ ]:
#seventh data quality issue: remove retweets 

In [ ]:
df_clean = df_clean[np.isnan(df_clean['retweeted_status_id'])]

In [ ]:
df_clean.drop(['retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp'],axis=1, inplace=True)

In [ ]:
#eighth data quality issue: drop reply columns

In [ ]:
df_clean.drop(['in_reply_to_user_id','in_reply_to_status_id'],axis=1, inplace=True)

## Test ##

In [ ]:
df3_clean.head(1)

# Tidyness #

## Define ##

•	The database had columns for different dog types. Since only one type can be selected, the others just have “None” values. To tidy the database, I merged the columns into one column called “dogs” and changed the values to reflect the selected breed without the “None” values. Then, I dropped the original columns.

•	Then, I merged all three databases using “tweet_id” column to have one master database that contains all the columns. Note that I merged the clean databases and not the original.


## Code ##

In [ ]:
#first tidyness issue: merge all dog columns into one

In [ ]:
df_clean['dogs'] = df_clean['doggo'] + df_clean['floofer'] + df_clean['pupper'] + df_clean['puppo']

In [ ]:
df_clean['dogs'].unique()

In [ ]:
df_clean.loc[df_clean['dogs'] == 'NoneNoneNoneNone',['dogs']] = 'None'
df_clean.loc[df_clean['dogs'] == 'doggoNoneNoneNone',['dogs']] = 'doggo'
df_clean.loc[df_clean['dogs'] == 'NoneNoneNonepuppo',['dogs']] = 'puppo'
df_clean.loc[df_clean['dogs'] == 'NoneNonepupperNone',['dogs']] = 'pupper'
df_clean.loc[df_clean['dogs'] == 'NoneflooferNoneNone',['dogs']] = 'floofer'
df_clean.loc[df_clean['dogs'] == 'doggoNoneNonepuppo',['dogs']] = 'doggo, puppo'
df_clean.loc[df_clean['dogs'] == 'doggoflooferNoneNone',['dogs']] = 'doggo, floofer'
df_clean.loc[df_clean['dogs'] == 'doggoNonepupperNone',['dogs']] = 'doggo, pupper'

In [ ]:
df_clean.drop(['puppo','pupper','floofer','doggo'],axis=1, inplace=True)

In [ ]:
df_clean['dogs'].value_counts()

In [ ]:
#second tidyness issue: merge all databases together

In [ ]:
df_all = pd.merge(df_clean,df2_clean, on ='tweet_id', how = 'inner')

In [ ]:
df_all = pd.merge(df_all,df3_clean, on='tweet_id', how='inner')

In [ ]:
df_all.head(3)

In [ ]:
for col in df_all.columns:
    print(col)

In [ ]:
df_all.drop(['source_y','possibly_sensitive','possibly_sensitive_appealable','in_reply_to_status_id','in_reply_to_status_id_str','in_reply_to_user_id','in_reply_to_user_id_str','in_reply_to_screen_name','retweeted_status','quoted_status_id','quoted_status_id_str','quoted_status','is_quote_status'],axis=1, inplace=True)

In [ ]:
for col in df_all.columns:
    print(col)

## Test ##

In [ ]:
df_all

In [ ]:
df_all.to_csv('Twitter_Archive_Master.csv',index=False)

# Insights #

In [ ]:
df_all.describe()

Three insights gained from looking at the master database:

    1- most denominators are 10 or above. Very few seem to fall below 10
    2- the language used the most in english
    3- there is a positive corralation between increasing likes and retweets

In [ ]:
df_all.rating_denominator.value_counts()

In [ ]:
df_all.lang.value_counts()

In [ ]:
plt.figure(figsize=(12,9))
plt.scatter(df_all['favorite_count'],df_all['retweet_count'],color = "blue")
plt.title('retweets vs favorites')
plt.xlabel('Number of favorites')
plt.ylabel('Number of retweets')
plt.show()